In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Restrict minor warnings
import warnings
warnings.filterwarnings('ignore')



In [ ]:



# Import test and train data
df_train = pd.read_csv('../input/forest-cover-type-prediction/train.csv')
df_Test = pd.read_csv('../input/forest-cover-type-prediction/test.csv')
df_test = df_Test

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.dtypes

All of the features in our datasets are numerical, except for Wilderness Area, Soil Type, and ****Cover Type**** 

In [ ]:
pd.set_option('display.max_columns', None) # we need to see all the columns
df_train.describe()

In [ ]:
# frequency count of column A 
count_train = df_train['Cover_Type'].value_counts() 
print(count_train)


In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of forest categories(Target Variable)")
ax = sns.distplot(df_train["Cover_Type"])

**Data Exploration results**
  
* each class is having 2160 entries and 
* The id is unique for each entry so it can be removed
*  Total 15120 entries and each column  is having the same number of entries, we can conclude no missing values 
* Each value of Soil_Type7 and Soil_Type15 are equal to zero, so we can remove these attributes.
*  Rescaling and normalization may need because the scales are different for different attribute 
*  Wilderness_Area[1-4] and Soil_Type[1-40] are one-hot encoded. 




In [ ]:
# Drop 'Id'
df_train = df_train.iloc[:,1:]
df_test = df_test.iloc[:,1:]



Removing ID,Soil_Type7 and Soil_Type15



In [ ]:
# Drop Soil_Type7 and Soil_Type15
df_train =df_train.drop(['Soil_Type7','Soil_Type15'], axis = 1)
df_test =df_test.drop(['Soil_Type7','Soil_Type15'], axis = 1)



Correlation analysis of countinous attributes using heatmap
* Ignore biary valued attributes like Wilderness_Area[1-4] and Soil_Type[1-40] for the correlation analysis
  




In [ ]:
no_of_continuous_attributes = 10
correlation_matrix =df_train.iloc[:,:no_of_continuous_attributes].corr()
f, ax = plt.subplots(figsize = (10,8))
sns.heatmap(correlation_matrix,vmax=0.8,square=True);



In [ ]:
#Correlation values

data = df_train.iloc[:,:no_of_continuous_attributes]






In [ ]:
# Get name of the continuous attributes
cols = data.columns
print(cols)

In [ ]:
# Calculate the pearson correlation coefficients for all combinations
data_corr = data.corr()

# Threshold ( only highly correlated ones matter)
threshold = 0.5
corr_list = []

In [ ]:
# Get correlation matrix
data_corr

In [ ]:
#Bubble sorting of correlation array
             
                
    for i in range(0, no_of_continuous_attributes):
        for j in range(i+1, no_of_continuous_attributes):
            if data_corr.iloc[i,j]>= threshold and data_corr.iloc[i,j]<1\
                or data_corr.iloc[i,j] <0 and data_corr.iloc[i,j]<=-threshold:
                    corr_list.append([data_corr.iloc[i,j],i,j])

In [ ]:
#sort the correlation values

s_corr_list = sorted(corr_list,key= lambda x: -abs(x[0]))

# print the higher values
for v,i,j in s_corr_list:
    print("%s and %s = %.2f" % (cols[i], cols[j], v))

**Skewness  of countinous attributes**


In [ ]:
df_train.iloc[:,:10].skew()

**Skewness is present in all the attribute  **

****Exploring the relationship between attributes****

In [ ]:
for v,i,j in s_corr_list:
    sns.pairplot(data = df_train, hue='Cover_Type', size= 10, x_vars=cols[i], y_vars=cols[j])
    plt.show()

* 'Horizontal_Distance_To_Hydrology' and  'Vertical_Distance_To_Hydrology' seems to have a linear relation
* Hillside and Aspect seems to have a sigmoid relation

**Violin Plot**
Violin Plot is a method to visualize the distribution of numerical data of different variables. It is similar to Box Plot but with a rotated plot on each side, giving more information about the density estimate on the y-axis.

In [ ]:
print(df_train.columns)

In [ ]:
columns = df_train.columns
#target value is not needed
size = len(columns)-1

# x-axis has target attributes to distinguish between classes
x = columns[size]
y = columns[0:size]

for i in range(0, size):
    sns.violinplot(data=df_train, x=x, y=y[i])
    plt.show()

****
* Elevation has a seperate distribution for each class, hence an important attribute for prediction
* Aspect plot contains couple of normal distribution for '1' and '6'classes
* 'Horizontal_Distance_To_Hydrology' and 'Horizontal_Distance_To_Roadways' are quite similar
* 'Hillshade_9am'and  'Hillshade_Noon'are left skewed
* In 'Wilderness_Area3' all the classes are distributed similarly
* 'Wilderness_Area1','Wilderness_Area2' and 'Wilderness_Area4' offer class distinction as values are not present for many classes
* Soil_Type, 1,5,8,9,12,14,18-22, 25-30 and 35-40 offer class distinction as values are not present for many classes

In [ ]:
print(df_train.columns)

In [ ]:
df_train.Wilderness_Area1.value_counts()

In [ ]:
df_train.Wilderness_Area2.value_counts()

In [ ]:
df_train.Wilderness_Area3.value_counts()

In [ ]:
df_train.Wilderness_Area4.value_counts()

In [ ]:
#grouping of one-hot encoded variables into a single variable
cols = df_train.columns
r,c = df_train.shape

In [ ]:
# Create a new dataframe with r rows, one column for each encoded category[Wilderness_Area(1-4),Soil_type[1-40], and target in the end
new_data = pd.DataFrame(index= np.arange(0,r), columns=['Wilderness_Area', 'Soil_Type', 'Cover_Type'])

In [ ]:

# Make an entry in data for each r for category_id, target_value
for i in range(0,r):
    p = 0;
    q = 0;
    # Category1_range
    for j in range(10,14):
        if (df_train.iloc[i,j] == 1):
            p = j-9 # category_class
            break
    # Category2_range
    for k in range(14,54):
        if (df_train.iloc[i,k] == 1):
            q = k-13 # category_class
            break
    # Make an entry in data for each r
    new_data.iloc[i] = [p,q,df_train.iloc[i, c-1]]

In [ ]:
# plot for category1
sns.countplot(x = 'Wilderness_Area', hue = 'Cover_Type', data = new_data)
plt.show()

* Wilderness_Area4 has lot of presence of cover_type 4, good class distinction



An attribut is having too many zero values means that shows class distiction because Wilderness_Area[1-4]are one hot encoded data. 

In [ ]:
# Plot for category2
plt.rc("figure", figsize = (25,10))
sns.countplot(x='Soil_Type', hue = 'Cover_Type', data= new_data)
plt.show()

SoilType 1-6,9-13,15, 20-22, 27-31,35,36-38 offer lot of class distinction as counts for some are very high. in all these one or two types of covertype is dominating .This type of attributes will help to classify the data .

In [ ]:
# calculating the count of soil_type[1-40] as well as  Wilderness_Area[1-4] 
for i in range(10,df_train.shape[1]-1):
    j = df_train.columns[i]
    print (df_train[j].value_counts())




* 'Soil_Type8'and  'Soil_Type25'having one count towrds 1. It wont contribute much for the classification

In [ ]:
# Let's drop them
df_train = df_train.drop(['Soil_Type8', 'Soil_Type25'], axis=1)
df_test = df_test.drop(['Soil_Type8', 'Soil_Type25'], axis=1)
df_train1 = df_train # To be used for algos like SVM where we need normalization and StandardScaler
df_test1 = df_test # To be used under normalization and StandardScaler

**Normality checking**
* Needed for a few algorithms like SVM
* Consider only non categorical values 



In [ ]:
df_train.iloc[:,:10].skew()

**Data transformation needed in: 'Horizontal_Distance_To_Hydrology,Horizontal_Distance_To_Roadways and Vertical_Distance_To_Hydrology  ', 'Hillshade_9am & Hillshade_noon'**

In [ ]:
#Horizontal_Distance_To_Hydrology
from scipy import stats
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Horizontal_Distance_To_Hydrology'], fit = stats.norm)

In [ ]:
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Horizontal_Distance_To_Hydrology'], plot=plt)

These plots shows the Horizontal_Distance_To_Hydrology is right/poitively skewed .It shows positive skewness (log or squared transformations will be a good option)


In [ ]:
df_train2=df_train1

In [ ]:
df_train1['Horizontal_Distance_To_Hydrology'] = np.sqrt(df_train1['Horizontal_Distance_To_Hydrology'])

In [ ]:
# Plot again after sqrt transformation
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Horizontal_Distance_To_Hydrology'], fit = stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Horizontal_Distance_To_Hydrology'], plot=plt)

In [ ]:
df_train1.head()

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Horizontal_Distance_To_Roadways'], fit = stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Horizontal_Distance_To_Roadways'], plot=plt)

In [ ]:
df_train1['Horizontal_Distance_To_Roadways'] = np.sqrt(df_train1['Horizontal_Distance_To_Roadways'])


In [ ]:
# Plot again after sqrt transformation
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Horizontal_Distance_To_Roadways'], fit = stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Horizontal_Distance_To_Roadways'], plot=plt)

In [ ]:
# Plot again after sqrt transformation
plt.figure(figsize=(8,6))
sns.distplot(df_train2['Horizontal_Distance_To_Roadways'], fit = stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train2['Horizontal_Distance_To_Roadways'], plot=plt)

In [ ]:
#Hillshade_9am
fig = plt.figure(figsize=(8,6))
sns.distplot(df_train1['Hillshade_9am'],fit=stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Hillshade_9am'],plot=plt)

****Negative skewness****

In [ ]:
df_train1['Hillshade_9am'] = np.square(df_train1['Hillshade_9am'])

In [ ]:
# Plot again after square transformation
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Hillshade_9am'], fit = stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Hillshade_9am'], plot=plt)

**Squared data transformation is good **

In [ ]:
# Hillshade_Noon
fig = plt.figure(figsize=(8,6))
sns.distplot(df_train1['Hillshade_Noon'],fit=stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Hillshade_Noon'],plot=plt)


Negative skewness

In [ ]:
df_train1.head()

In [ ]:
df_train1['Hillshade_Noon'] = np.square(df_train1['Hillshade_Noon'])

In [ ]:
# Hillshade_Noon
fig = plt.figure(figsize=(8,6))
sns.distplot(df_train1['Hillshade_Noon'],fit=stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Hillshade_Noon'],plot=plt)

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Horizontal_Distance_To_Fire_Points'], fit=stats.norm)
plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Horizontal_Distance_To_Fire_Points'],plot=plt)

In [ ]:
df_train1['Horizontal_Distance_To_Fire_Points'] = np.sqrt(df_train1['Horizontal_Distance_To_Fire_Points'])

In [ ]:
# Plot again after sqrt transformation
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Horizontal_Distance_To_Fire_Points'], fit=stats.norm)
plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Horizontal_Distance_To_Fire_Points'],plot=plt)

In [ ]:
#Vertical_Distance_To_Hydrology
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Vertical_Distance_To_Hydrology'], fit = stats.norm)
fig = plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Vertical_Distance_To_Hydrology'], plot=plt)

In [ ]:
plt.figure(figsize=(8,6))
sns.distplot(df_train1['Vertical_Distance_To_Hydrology'], fit=stats.norm)
plt.figure(figsize=(8,6))
res = stats.probplot(df_train1['Vertical_Distance_To_Hydrology'],plot=plt)

In [ ]:
# performing same transformation in test dataset 
df_test1[['Horizontal_Distance_To_Hydrology','Horizontal_Distance_To_Fire_Points'\
        ,'Horizontal_Distance_To_Roadways']] = np.sqrt(df_test1[['Horizontal_Distance_To_Hydrology',\
        'Horizontal_Distance_To_Fire_Points','Horizontal_Distance_To_Roadways']])

In [ ]:
df_test1[['Hillshade_9am','Hillshade_Noon']] = np.square(df_test1[['Hillshade_9am','Hillshade_Noon']])

******Training and Testing  Data******

**Preprocessing**

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Taking only non-categorical values
Size = 10
X_temp = df_train.iloc[:,:Size]
X_test_temp = df_test.iloc[:,:Size]
X_temp1 = df_train1.iloc[:,:Size]
X_test_temp1 = df_test1.iloc[:,:Size]
a = df_train.iloc[:,:Size]


X_temp1 = StandardScaler().fit_transform(X_temp1)
X_test_temp1 = StandardScaler().fit_transform(X_test_temp1)

In [ ]:
r,c = df_train.shape
print(df_train.shape)

In [ ]:
r,c = df_train.shape
X_train = np.concatenate((X_temp,df_train.iloc[:,Size:c-1]),axis=1)
X_train1 = np.concatenate((X_temp1, df_train1.iloc[:,Size:c-1]), axis=1) # to be used for SVM
y_train = df_train.Cover_Type.values

******Support Vector Machine******

In [ ]:
from sklearn import svm as svm
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
# Setting parameters
x_data, x_test_data, y_data, y_test_data = train_test_split(X_train1,y_train,test_size=0.2, random_state=123)
svm_para = [{'kernel':['rbf'],'C': [1,10,100,100]}]

In [ ]:
# Parameters optimized using the code in above cell
C_opt = 10 # reasonable option
clf = svm.SVC(C=C_opt,kernel='rbf')
clf.fit(X_train1,y_train)

In [ ]:
clf.score(X_train1,y_train)

In [ ]:
#y_pred = clf.predict(X_test1)

****ExtraTreesClassifier****

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report

In [ ]:
# setting parameters
x_data, x_test_data, y_data, y_test_data = train_test_split(X_train,y_train,test_size= 0.3, random_state=0)
etc_para = [{'n_estimators':[20,30,100], 'max_depth':[5,10,15], 'max_features':[0.1,0.2,0.3]}] 

In [ ]:
#ETC = GridSearchCV(ExtraTreesClassifier(),param_grid=etc_para, cv=3, n_jobs=-1)
#ETC.fit(x_data, y_data)
#ETC.best_params_
#ETC.grid_scores_

In [ ]:
# setting parameters
x_data, x_test_data, y_data, y_test_data = train_test_split(X_train,y_train,test_size= 0.3, random_state=0)
etc_para = [{'n_estimators':100, 'max_depth':15, 'max_features':0.3}] 
# Default number of features is sqrt(n)
# Default number of min_samples_leaf is 1

In [ ]:
# Classification Report
Y_pred = ETC.predict(x_test_data)
target = ['class1', 'class2','class3','class4','class5','class6','class7' ]
print (classification_report(y_test_data, Y_pred, target_names=target))

****Learning Curve for Extra Trees Classifier****

In [ ]:
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
def plot_learning_curve(model,title, X, y,n_jobs = 1, ylim = None, cv = None,train_sizes = np.linspace(0.1, 1, 5)):
    
    # Figrue parameters
    plt.figure(figsize=(10,8))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel('Training Examples')
    plt.ylabel('Score')
    
    train_sizes, train_score, test_score = learning_curve(model, X, y, cv = cv, n_jobs=n_jobs, train_sizes=train_sizes)

    # Calculate mean and std
    train_score_mean = np.mean(train_score, axis=1)
    train_score_std = np.std(train_score, axis=1)
    test_score_mean = np.mean(test_score, axis=1)
    test_score_std = np.std(test_score, axis=1)
    
    plt.grid()
    plt.fill_between(train_sizes, train_score_mean - train_score_std, train_score_mean + train_score_std,\
                    alpha = 0.1, color = 'r')
    plt.fill_between(train_sizes, test_score_mean - test_score_std, test_score_mean + test_score_std,\
                    alpha = 0.1, color = 'g')
    
    plt.plot(train_sizes, train_score_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_score_mean, 'o-', color="g", label="Cross-validation score")
    
    plt.legend(loc = "best")
    return plt


In [ ]:
# 'max_features': 0.3, 'n_estimators': 100, 'max_depth': 15, 'min_samples_leaf: 1'
etc = ExtraTreesClassifier(bootstrap=True, oob_score=True, n_estimators=100, max_depth=10, max_features=0.3, \
                           min_samples_leaf=1)

etc.fit(X_train, y_train)
# yy_pred = etc.predict(X_test)
etc.score(X_train, y_train)

In [ ]:
# Plotting learning curve
title = 'Learning Curve (ExtraTreeClassifier)'
# cross validation with 50 iterations to have a smoother curve
cv = ShuffleSplit(n_splits=50, test_size=0.2, random_state=0)
model = etc
plot_learning_curve(model,title,X_train, y_train, n_jobs=-1,ylim=None,cv=cv)
plt.show()